In [ ]:
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))

In [ ]:
import pandas as pd
import numpy as np
from os.path import join

from datetime import datetime, date, timedelta
import random



from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

random.seed(10)

In [ ]:
testes_previstos = 100000
caixas_previstas = testes_previstos / 20
print(testes_previstos, caixas_previstas)

In [ ]:
planilha = pd.read_excel('01RS.xlsx')
planilha

In [ ]:
planilha['% DA POP TOTAL (VERAO)'] = planilha['POPULACAO_VERAO'] / planilha['POPULACAO_VERAO'].sum()
planilha

In [ ]:
planilha['CAIXAS'] = np.floor(planilha['% DA POP TOTAL (VERAO)'] * caixas_previstas)
planilha

In [ ]:
total = planilha['CAIXAS'].sum()

In [ ]:
testes_randomicos = random.choices(planilha['MUNICIPIO'], weights=planilha['% DA POP TOTAL (VERAO)'], k=int(caixas_previstas - total))
planilha = planilha.join(pd.Series(testes_randomicos, name='CAIXAS_ADICIONAIS').value_counts(), on='MUNICIPIO')
planilha

In [ ]:
planilha['CAIXAS'] = planilha['CAIXAS'] + planilha['CAIXAS_ADICIONAIS'].fillna(0)
del(planilha['CAIXAS_ADICIONAIS'])
planilha

In [ ]:
planilha['TESTES PREVISTOS'] = planilha['CAIXAS'] * 20
planilha.loc['TOTAL', ['POPULACAO', 'CASOS CONFIRMADOS', 'CAIXAS', 'TESTES PREVISTOS']] = planilha[['POPULACAO', 'CASOS CONFIRMADOS', 'CAIXAS', 'TESTES PREVISTOS']].sum()
planilha

In [ ]:
planilha.to_excel('testes_previstos_projeto_verao.xlsx')